In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langchain_community.llms.ollama import Ollama

llm = Ollama(model="dolphin-mistral")

llm.invoke("how can langsmith help with testing?")

"Langsmith is an artificial intelligence that assists in language generation and translation tasks. Although it doesn't directly aid with software testing, Langsmith can be integrated into the testing process to facilitate better communication and documentation. Here are some ways in which Langsmith can support the testing activities:\n\n1. Documentation: Langsmith can help generate or translate technical documentation related to test cases, test scenarios, and user stories. This can improve understanding among team members who speak different languages.\n\n2. Test reports: Langsmith can be used to translate test report summaries, allowing testers and stakeholders from diverse linguistic backgrounds to access the same information in their preferred language.\n\n3. Automated feedback: If Langsmith is integrated with an automated testing system, it could provide automated feedback for users who don't speak the software's primary language. For example, it can automatically translate error

## LLM Chain

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are world class technical documentation writer."),
        ("user", "{input}"),
    ]
)

chain = prompt | llm | output_parser

chain.invoke({"input": "how can langsmith help with testing?"})

'Dolphin: As a Langsmith, I can assist with testing by creating detailed and concise test plans, test cases, and user manuals. These documents can guide the development team to create robust, functional, and usable software. My expertise in technical documentation also allows me to provide clear instructions for testers to follow when executing tests and document their findings. Additionally, I can analyze test results and generate comprehensive reports that highlight any issues or areas for improvement. This way, my assistance helps ensure the quality of the product and contributes to its successful launch.<|im_end|>'

## Retriever

In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

## Embedding

In [8]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

## Vector Store

In [9]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

## Retrieval Chain

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

document_chain = create_stuff_documents_chain(llm, prompt)

from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can assist with testing through several methods, including creating datasets for running tests, providing a user-friendly comparison view to track and diagnose regressions in test scores across multiple revisions of an application, allowing users to attach feedback scores to logged traces, enabling developers to collect more data on how their LLM applications are performing in real-world scenarios through beta testing, and supporting threads to group traces from a single conversation together. This makes it easier to track the performance of and annotate your application across multiple turns.<|im_end|>


## Conversation Retrieval Chain

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""
)

document_chain = create_stuff_documents_chain(llm, prompt)

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        (
            "user",
            "Given the above conversation, generate a search query to look up to get information relevant to the conversation",
        ),
    ]
)
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes!"),
]
retriever_chain.invoke({"chat_history": chat_history, "input": "Tell me how"})

[Document(page_content='meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them somethin

## Agent

In [12]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.tools.google_search import GoogleSearchResults

search = GoogleSearchResults(api_wrapper=GoogleSearchAPIWrapper())

tools = [retriever_tool, search]

In [23]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_openai import ChatOpenAI
import os

prompt = hub.pull("hwchase17/openai-functions-agent")

llm = ChatOpenAI(
    model="gpt-3.5-turbo", temperature=0, base_url=os.getenv("OPENAI_API_URL")
)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})

In [35]:
agent_executor.invoke({"input": "what is the weather in Toronto?"})



> Entering new AgentExecutor chain...

Invoking: `google_search_results_json` with `weather in Toronto`


[{'title': '10-Day Weather Forecast for Midtown, Ontario, Canada - The ...', 'link': 'https://weather.com/weather/tenday/l/Midtown+Ontario+Canada?canonicalCityId=4d9dd70a933d0d5d4c894283242ddf650e6b0903fedd76d42e55803cb628e1e7', 'snippet': 'Be prepared with the most accurate 10-day forecast for Midtown, Ontario, Canada with highs, lows, chance of precipitation from The Weather Channel and\xa0...'}, {'title': 'Toronto, ON - 7 Day Forecast - Environment Canada', 'link': 'https://weather.gc.ca/city/pages/on-143_metric_e.html', 'snippet': "Weather Radar · Satellite · Lightning. Light Rain. 5° C. °C; °F. Observed at: Toronto Pearson Int'l Airport; Date: 8:12 AM EDT Saturday 13 April 2024. Condition\xa0..."}, {'title': 'The Weather Network - Toronto', 'link': 'https://www.theweathernetwork.com/en/city/ca/ontario/toronto/current', 'snippet': 'Get Toronto, ON. current weather report with

{'input': 'what is the weather in Toronto?',
 'output': 'I found some sources where you can check the weather in Toronto:\n1. [The Weather Channel - 10-Day Weather Forecast for Midtown, Ontario, Canada](https://weather.com/weather/tenday/l/Midtown+Ontario+Canada?canonicalCityId=4d9dd70a933d0d5d4c894283242ddf650e6b0903fedd76d42e55803cb628e1e7)\n2. [Environment Canada - Toronto, ON 7-Day Forecast](https://weather.gc.ca/city/pages/on-143_metric_e.html)\n3. [The Weather Network - Toronto](https://www.theweathernetwork.com/en/city/ca/ontario/toronto/current)\n4. [AccuWeather - Toronto, Ontario, Canada Weather Forecast](https://www.accuweather.com/en/ca/toronto/m5h/weather-forecast/55488)\n\nYou can visit these websites to get the current weather information for Toronto.'}

In [36]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith LLM application testing services'}`


meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright © 2024 LangChain, Inc.

LangSmith User Guide | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’l

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith provides a platform for LLM application development, monitoring, and testing. They offer various features and workflows to support different stages of the application development lifecycle, including prototyping, debugging, initial test set creation, comparison view, playground for rapid iteration, beta testing, capturing feedback, annotating traces, adding runs to datasets, production monitoring, A/B testing, automations, and threads for multi-turn applications. LangSmith allows developers to create datasets, run tests on LLM applications, track performance metrics, and automate actions on traces in near real-time.'}